# 300 Maximum Performance of a Team
## 题目描述
<https://leetcode-cn.com/problems/maximum-performance-of-a-team/>

公司有编号为 $1$ 到 $n$ 的 $n$ 个工程师，给你两个数组 $speed$ 和 $efficiency$ ，其中 $speed[i]$ 和 $efficiency[i]$ 分别代表第 $i$ 位工程师的速度和效率。请你返回由最多 $k$ 个工程师组成的 ​​​​​​最大团队表现值 ，由于答案可能很大，请你返回结果对 $10^9 + 7$ 取余后的结果。

团队表现值的定义为：一个团队中「所有工程师速度的和」乘以他们「效率值中的最小值」。

示例 1：
```
输入：n = 6, speed = [2,10,3,1,5,8], efficiency = [5,4,3,9,7,2], k = 2
输出：60
解释：
我们选择工程师 2（speed=10 且 efficiency=4）和工程师 5（speed=5 且 efficiency=7）。他们的团队表现值为 performance = (10 + 5) * min(4, 7) = 60 。
```

示例 2：
```
输入：n = 6, speed = [2,10,3,1,5,8], efficiency = [5,4,3,9,7,2], k = 3
输出：68
解释：
此示例与第一个示例相同，除了 k = 3 。我们可以选择工程师 1 ，工程师 2 和工程师 5 得到最大的团队表现值。表现值为 performance = (2 + 10 + 5) * min(5, 4, 7) = 68 。
```

示例 3：
```
输入：n = 6, speed = [2,10,3,1,5,8], efficiency = [5,4,3,9,7,2], k = 4
输出：72
```

提示：
* $1 <= n <= 10^5$
* $speed.length == n$
* $efficiency.length == n$
* $1 <= speed[i] <= 10^5$
* $1 <= efficiency[i] <= 10^8$
* $1 <= k <= n$


## 思路 贪心+堆。

首先按照 $efficicecy$ 从大到小排序（排序后工程师数组记为 $team$ ），然后维护一个大小为 $k$ 最小堆，和堆对应的元素速度和 $speedSum$，设置一个变量 $maxPerf$ 表示当前的全局最优解。排序好的元素从左到右遍历，如果堆的实际大小小于 $k$，直接将元素压入堆，如果大小已经到 $k$，那么如果当前元素的 $speed$ 大于堆顶元素的 $speed$，堆顶出堆当前元素入堆。每次入堆都要更新 $speedSum$，重新计算团队表现值，和当前最优解比对，更新当前最优解 $maxPerf$。

当遍历到元素 $team[i]$ 时，最小堆记录的其实是比当前元素 $efficicecy$ 大的前 $k$ 大 $speed$ 的组合。我们证明为什么这样遍历更新之后最大团队表现值是被搜索到的了，首先团队表现值只有在每次堆变化之后才会更新 $maxPerf$，用归纳法证明，遍历到 $team[i]$ 的时候，$maxPerf$ 记录的就是 $team[:i+1]$ 的最大团队表现值：
* 当 $i=0$ 时更新了 $maxPerf$，此时 $maxPerf$ 就是 $team[:1]$ 的最大团队表现值；
* 当假设当 $i=1..m-1$ 时，$maxPerf$ 是 $team[:i+1]$ 的最大团队表现值。当 $i=m$时，此时 $team[:m+1]$ 的最大团队表现值无非包含两种情况，就是包含 $m$ 和不包含 $m$ ：
    * 如果包含 $m$，因为此时 $team[m]$ 的 $efficiency$ 最小的，那么 $team[m]$ 一定是在前 $k$ 大的 $speed$ 的组合里，即 $m$ 会入堆，而且此时更新的 $maxPerf$ 是最优解。
    * 如果不包含 $m$，那么 $team[:m+1]$ 的最大团队表现值其实是由 $team[:m]$ 组成的，根据我们的归纳假设，$maxPerf$ 也是 $team[:m+1]$ 的最优解。此时 $m$ 可能入堆也可能不入堆，不过对于我们的最优解没有影响。
* 综上，即可证明最后 $maxPerf$ 即为最大团队表现值。

In [ ]:
# Python实现
class Solution:
    def maxPerformance(self, n: int, speed: list, efficiency: list, k: int) -> int:
        MOD = int(10 ** 9 + 7)
        team = [[speed[i], efficiency[i]] for i in range(n)]
        team.sort(key=lambda x: (x[1], x[0]), reverse=True)
        from queue import PriorityQueue
        pq = PriorityQueue()
        speedSum = 0
        maxPerf = 0

        for i in range(n):
            if i < k:
                pq.put(team[i][0])
                speedSum += team[i][0]
                maxPerf = max([maxPerf, speedSum * team[i][1]])
            else:
                if team[i][0] > pq.queue[0]:
                    speed_ = pq.get()
                    pq.put(team[i][0])
                    speedSum = speedSum - speed_ + team[i][0]
                    maxPerf = max([maxPerf, speedSum * team[i][1]])

        return maxPerf % MOD